# MINADOS:

## 1.Similitud de platos según ingredientes (Node Similarity)

In [ ]:
query = """
MATCH (m1:Meal)-[r:SIMILAR_TO]->(m2:Meal)
RETURN m1.name AS source, m2.name AS target, r.score AS weight
"""

with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]

import pandas as pd
df = pd.DataFrame(data)
df.to_csv("meal_similarity.csv", index=False)


En este primer minado aplicamos el algoritmo Node Similarity de Neo4j sobre los platos, utilizando los ingredientes compartidos como base. Esto nos permitió establecer relaciones directas entre platos similares, generando un grafo de similitud donde cada conexión representa un cierto grado de coincidencia culinaria. A partir de esta red, se pueden realizar recomendaciones de platos parecidos o encontrar variaciones basadas en ingredientes comunes. Esta estructura fue visualizada y explorada mediante Pyvis para identificar patrones temáticos como postres, ensaladas o platos principales.

Node Similarity no agrupa nodos en comunidades, sino que calcula una puntuación de similitud entre pares de nodos en función de los elementos que comparten. En este caso, la similitud entre platos se basa en los ingredientes en común, permitiendo establecer relaciones directas que pueden utilizarse para recomendaciones o análisis de variantes.

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from neo4j import GraphDatabase

# Conectar a Neo4j
driver = GraphDatabase.driver(URI, auth=(USER, PASSWORD))

# Consulta Cypher para sacar relaciones SIMILAR_TO con score alto
query = """
MATCH (m1:Meal)-[r:SIMILAR_TO]->(m2:Meal)
WHERE r.score > 0.6
RETURN m1.name AS source, m2.name AS target, r.score AS weight
"""

# Ejecutar y cargar en DataFrame
with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]
df = pd.DataFrame(data)

# Crear grafo
G = nx.from_pandas_edgelist(df, source='source', target='target', edge_attr='weight')

# Dibujar
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(G, k=0.5)
edges = G.edges(data=True)
weights = [d['weight'] * 5 for _, _, d in edges]

nx.draw(G, pos, with_labels=True, node_size=500, font_size=8, edge_color=weights, width=2.0, edge_cmap=plt.cm.Blues)
plt.title("Platos similares según ingredientes")
plt.show()


### Ahora, si usamos pyvis, podremos generar un gráfico más interactivo

In [ ]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from neo4j import GraphDatabase

# Conexión con Neo4j
driver = GraphDatabase.driver("bolt://localhost:7689", auth=("neo4j", "admin123"))

# Consulta
query = """
MATCH (m1:Meal)-[r:SIMILAR_TO]->(m2:Meal)
WHERE r.score > 0.6
RETURN m1.name AS source, m2.name AS target, r.score AS weight
"""

# Obtener datos
with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]
df = pd.DataFrame(data)

# Crear grafo desde pandas (networkx)
G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr='weight')

# Crear visualización interactiva
net = Network(height="750px", width="100%", notebook=False)
net.add_nodes(G.nodes)

# Agregar edges con atributos explícitos
for u, v, d in G.edges(data=True):
    net.add_edge(u, v, value=d['weight'], title=f"Similitud: {d['weight']:.2f}")

# Opcional: agregar títulos a los nodos
for node in net.nodes:
    node['title'] = node['label']
    node['value'] = 1

# Guardar resultado
net.write_html("similitud_meals.html")
#net.show("similitud_meals.html")


## 2. Detección de comunidades de comidas (Louvain)

Con el algoritmo de Louvain agrupamos automáticamente los platos en comunidades, basándonos en sus similitudes. Mientras que en el minado anterior conectamos cada plato con otros según su similitud (uno a uno), ahora identificamos grupos cohesivos de platos que están densamente conectados entre sí. Esto nos permite cuantificar y visualizar los clústeres naturales dentro del conjunto de recetas, no solo pares similares.

Al tratarse de un algoritmo de aprendizaje no supervisado, el modelo no asigna un nombre a cada grupo, sino que simplemente detecta conjuntos de nodos que están densamente conectados entre sí. La interpretación de esos grupos queda a criterio del analista.

En nuestro enfoque, el algoritmo Node Similarity se utiliza como paso previo a la detección de comunidades. Su función es generar un grafo de similitudes entre platos, calculando una puntuación para cada par en función de los ingredientes que comparten. A partir de este grafo, se aplica el algoritmo Louvain, que permite identificar grupos de platos densamente conectados entre sí. De este modo, la similitud entre recetas sirve como base estructural sobre la que Louvain puede detectar comunidades naturales, sin necesidad de etiquetas o clasificaciones previas.

In [ ]:
import pandas as pd
import networkx as nx
from pyvis.network import Network
from neo4j import GraphDatabase
import matplotlib.pyplot as plt
import seaborn as sns

# Conexión con Neo4j
driver = GraphDatabase.driver("bolt://localhost:7689", auth=("neo4j", "admin123"))

# Consulta de Meals con comunidades y similitud
query = """
MATCH (m1:Meal)-[r:SIMILAR_TO]->(m2:Meal)
WHERE m1.community IS NOT NULL AND m2.community IS NOT NULL
RETURN m1.name AS source, m2.name AS target, r.score AS weight, m1.community AS community1, m2.community AS community2
"""

# Ejecutar y cargar
with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]
df = pd.DataFrame(data)

# Crear grafo con comunidad como grupo
G = nx.from_pandas_edgelist(df, 'source', 'target', edge_attr='weight')
comunidades = {}
for _, row in df.iterrows():
    comunidades[row['source']] = row['community1']
    comunidades[row['target']] = row['community2']

net = Network(height="750px", width="100%")
for node in G.nodes:
    net.add_node(node, label=node, group=comunidades.get(node, 0))

for u, v, d in G.edges(data=True):
    net.add_edge(u, v, value=d['weight'], title=f"Similitud: {d['weight']:.2f}")

#net.show("meal_communities.html")
net.write_html("meal_communities.html")


In [ ]:
# Distribución de comidas por comunidad
community_counts = pd.Series(list(comunidades.values())).value_counts().sort_index()
plt.figure(figsize=(10, 6))
sns.barplot(x=community_counts.index, y=community_counts.values, palette="tab10", hue=community_counts.index, legend=False)
plt.xlabel("ID de comunidad")
plt.ylabel("Número de comidas")
plt.title("Distribución de Meals por Comunidad (Louvain)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 3. Centralidad de ingredientes

En el tercer minado nos centramos en identificar los ingredientes más relevantes dentro de la red de recetas. Para ello, aplicamos el algoritmo PageRank sobre el grafo bipartito de platos e ingredientes (tipo de grafo en el que los elementos de un mismo tipo no se pueden relacionar entre sí). Este análisis permite detectar cuáles son los ingredientes más influyentes, no solo por su frecuencia, sino también por aparecer en recetas que a su vez están conectadas con otros ingredientes importantes. El resultado se representó con un gráfico de barras que muestra los ingredientes más centrales del ecosistema culinario.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from neo4j import GraphDatabase

# Conexión con Neo4j
driver = GraphDatabase.driver("bolt://localhost:7689", auth=("neo4j", "admin123"))

# Consulta Cypher para obtener PageRank de ingredientes
query = """
MATCH (i:Ingredient)
WHERE i.pagerank IS NOT NULL
RETURN i.name AS ingredient, i.pagerank AS score
ORDER BY score DESC
LIMIT 20
"""

# Ejecutar y cargar en DataFrame
with driver.session() as session:
    result = session.run(query)
    data = [record.data() for record in result]
df = pd.DataFrame(data)

# Visualizar con seaborn
plt.figure(figsize=(12, 6))
sns.barplot(x="score", y="ingredient", data=df, palette="viridis", legend=False, hue="ingredient")
plt.xlabel("PageRank")
plt.ylabel("Ingrediente")
plt.title("Top 20 Ingredientes más influyentes según PageRank")
plt.tight_layout()
plt.show()
